<a href="https://colab.research.google.com/github/naitotomoyuki/2021-singakujutu/blob/master/ROCexample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: 最初の列は0から始まるデータ番号であり、最初リストは全て0、２番目のリストは１となる。２列目は数値
# 出力はoutput.xlsxファイルの保存されるので必要であればDLする
#　これはシミュレーションデータ生成コードなので、実データがある場合は実行せず、代わりに実データをoutput.xlsxとしてアップロードする
import numpy as np
import pandas as pd


def generate_normal_data(mean, variance, num_data):
  """
  平均値と分散を指定し、正規分布から指定された数のデータを作成する関数。

  Args:
    mean: 平均値。
    variance: 分散。
    num_data: 作成するデータの数。

  ただし０より小さいデータは０、
  １より大きいデータは１とする簡易的なデータ生成

  Returns:
    指定された正規分布から生成されたデータのリスト。
  """
  std_dev = np.sqrt(variance)
  data = np.random.normal(loc=mean, scale=std_dev, size=num_data)
  data[data < 0] = 0
  data[data > 1] = 1
  return data

# 正常データの平均値、分散、データ数を設定
mean_normal = 0.275
variance_normal = 0.085
num_data_normal = 100

# 異常データの平均値、分散、データ数を設定
mean_anormal = 0.557
variance_anormal = 0.148
num_data_anormal = 100

# データを作成
data1 = generate_normal_data(mean_anormal, variance_anormal, num_data_anormal)
data2 = generate_normal_data(mean_normal, variance_normal, num_data_normal)

# データ番号のリストを作成
data_number = list(range(num_data_normal+num_data_anormal))
list_0 = [0] * num_data_anormal
list_1 = [1] * num_data_normal

true = np.concatenate((list_0, list_1))
pred = np.concatenate((data1, data2))

# DataFrameを作成
df = pd.DataFrame({
    'True': true,
    'Pred': pred
})

# Excelファイルとして保存
df.to_excel('output.xlsx', index=False)

In [ ]:
# prompt: output.xlsファイルを読み込み、Trueの値が０のデータと1のデータの分布を重ねてグラフ化する.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Excelファイルを読み込む
df = pd.read_excel('output.xlsx')

# Trueの値が0のデータと1のデータを取得
df_true_0 = df[df['True'] == 0]
df_true_1 = df[df['True'] == 1]

# ヒストグラムを作成
plt.hist(df_true_0['Pred'], bins=20, alpha=0.5, label='anormal')
plt.hist(df_true_1['Pred'], bins=20, alpha=0.5, label='normal')

# グラフのタイトル、ラベルを設定
plt.xlabel('Pred', fontsize=18)
plt.ylabel('Frequency', fontsize=18)
plt.title('Distribution of anormal index', fontsize=24)

# 凡例を表示
plt.legend(fontsize=12)

# メモリの数値のフォントサイズを設定
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# グラフを表示
plt.show()

In [ ]:
# prompt: output.xlsxを読み込みROC曲線をかく

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Excelファイルを読み込む
df = pd.read_excel('output.xlsx')

# TrueとPredの値を取得
y_true = df['True']
y_scores = df['Pred']

# ROC曲線を作成
fpr, tpr, thresholds = roc_curve(y_true, 1-y_scores)

# AUCを計算
roc_auc = auc(fpr, tpr)

# ROC曲線をプロット
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=24)
plt.ylabel('True Positive Rate', fontsize=24)
plt.title('ROC of anormal detection', fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(loc="lower right")
plt.show()